In [1]:
input_file = "input_files/day_24.txt"

with open(input_file) as lines:
    data = lines.read().splitlines()
    

In [2]:
import numpy as np

def parse_data(data):
    result = []
    for line in data:
        pos, vel = line.split(' @ ')
        pos = tuple(map(int, pos.split(', ')))
        vel = tuple(map(int, vel.split(', ')))
        result.append((pos, vel))
    return result

d = np.array(parse_data(data), dtype=np.longlong)


## Part One
Small enough to brute force the combinations

In [3]:
from itertools import combinations

bounds_min = 200000000000000
bounds_max = 400000000000000
count = 0

for a, b in combinations(d[:,:,:2], r=2):
    '''
    a and b in the form of 
    [point], [vector]
    '''
    vector = np.column_stack((a[1], - b[1]))
    points = b[0] - a[0]

    # check if parallel
    if np.linalg.matrix_rank(vector) == 2:
       
        t = np.linalg.solve(vector, points)

        # where line a is at time t
        intersection_point = a[0] + t[0] * a[1] 

        in_time = (t > 0).all()
        in_box = np.logical_and(intersection_point >= bounds_min, intersection_point <= 400000000000000).all()

        if in_box and in_time:
            count+= 1

count

12740

## Part Two:

🤮 Manually build a system of equations to hand to linalg.solve. There must be a better way.

$(x,y,z) + t(x^\prime,y^\prime, z^\prime) = (x_0,y_0,z_0) + t(x_0^\prime,y_0^\prime, z_0^\prime)$

$(x,y,z) - (x_0,y_0,z_0) =  t((x_0^\prime,y_0^\prime, z_0^\prime) - (x^\prime,y^\prime, z^\prime))$

In [4]:

def part2(lines):    
    
    diffs_0_1 = lines[1] - lines[0] 
    diffs_0_2 = lines[2] - lines[0]


    a_directions = np.array(
     [
         [ diffs_0_1[1][1],   -diffs_0_1[1][0],  0,               ],
         [ diffs_0_2[1][1],   -diffs_0_2[1][0],  0,               ],
         [ diffs_0_1[1][2],   0,                 -diffs_0_1[1][0] ],
         [ diffs_0_2[1][2],   0,                 -diffs_0_2[1][0] ],
         [ 0,                 diffs_0_1[1][2],   -diffs_0_1[1][1] ],
         [ 0,                 diffs_0_2[1][2],   -diffs_0_2[1][1] ]
     ])
    
    a_positions = np.array([
        [ -diffs_0_1[0][1],  diffs_0_1[0][0],  0               ],
        [ -diffs_0_2[0][1],  diffs_0_2[0][0],  0               ],
        [ -diffs_0_1[0][2],  0,                diffs_0_1[0][0] ],
        [ -diffs_0_2[0][2],  0,                diffs_0_2[0][0] ],
        [   0,               -diffs_0_1[0][2], diffs_0_1[0][1] ],
        [   0,               -diffs_0_2[0][2], diffs_0_2[0][1] ],
        ])
    
    A = np.hstack([a_directions, a_positions])

    #d _ p
    x_y = d[:3,1,0] * d[:3,0,1] 
    x_z = d[:3,1,0] * d[:3,0,2] 
    y_z = d[:3,1,1] * d[:3,0,2] 
    y_x = d[:3,1,1] * d[:3,0,0] 
    
    # p _ d
    z_x = d[:3,0,0] * d[:3,1,2]
    z_y = d[:3,0,1] * d[:3,1,2]
    
    x = [
        (x_y[0] - x_y[1]) - (y_x[0] - y_x[1]),
        (x_y[0] - x_y[2]) - (y_x[0] - y_x[2]),
        (x_z[0] - x_z[1]) - (z_x[0] - z_x[1]),
        (x_z[0] - x_z[2]) - (z_x[0] - z_x[2]),
        (y_z[0] - y_z[1]) - (z_y[0] - z_y[1]),
        (y_z[0] - y_z[2]) - (z_y[0] - z_y[2]),
    ]
    
    res = np.linalg.solve(A, x)
    print([round(c) for c in res])
    return np.ceil(res[:3].sum())


print(part2(d[:3]))

[131633231355646, 371683716481155, 238674624073734, 268, -197, 68]
741991571910536.0


## Will sympy just do this for us? yes...🤦

In [5]:
import sympy

pos_x, pos_y, pos_z, dir_x, dir_y, dir_z = sympy.symbols("pos_x, pos_y, pos_z, dir_x, dir_y, dir_z")

eq_components = []

for i, ((x, y, z), (vx, vy, vz)) in enumerate(d[:3]):
    eq_components.append((pos_y - y) * (vz - dir_z) - (pos_z - z) * (vy - dir_y))
    eq_components.append((pos_x - x) * (vy - dir_y) - (pos_y - y) * (vx - dir_x))

answer = next(ans for ans in sympy.solve(eq_components) 
               if all(x.is_integer for x in ans.values())
              )
    

print(answer[pos_x] + answer[pos_y] + answer[pos_z])


741991571910536
